In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [3]:
mlb_df = pd.read_csv("assets/mlb.csv")

In [4]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [5]:
cities = pd.read_html("assets/wikipedia_data.html")[1]

In [6]:
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [7]:
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [8]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [9]:
def clear_data(string1):
    if re.search(r"\[[a-z]* [0-9]+\]",string1) is None:
        return string1
    else:
        return string1.replace(re.search(r"\[[a-z]* [0-9]+\]",string1).group(),"")

In [10]:
cities["MLB"] = cities["MLB"].apply(lambda x: clear_data(x))

In [11]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets,Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [13]:
mlb_cities = cities[["Metropolitan area","MLB"]].set_index("MLB")

In [14]:
mlb_cities.head()

,Metropolitan area
MLB,
Yankees Mets,New York City
Dodgers Angels,Los Angeles
Giants Athletics,San Francisco Bay Area
Cubs White Sox,Chicago
Rangers,Dallas–Fort Worth


In [15]:
mlb_cities = mlb_cities.drop(["—",""],axis = 0)

In [16]:
mlb_df = mlb_df[mlb_df["year"]==2018]

In [17]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [18]:
population = cities[["Metropolitan area","Population (2016 est.)[8]"]].set_index("Metropolitan area")

In [19]:
population.head()

,Population (2016 est.)[8]
Metropolitan area,
New York City,20153634
Los Angeles,13310447
San Francisco Bay Area,6657982
Chicago,9512999
Dallas–Fort Worth,7233323


In [21]:
mlb_df = mlb_df.copy()

In [22]:
mlb_df["area"] =mlb_df["team"].apply(lambda x: x.split(" ")[-1])

In [23]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League,area
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Sox
1,New York Yankees,100,62,0.617,8.0,2018,MLB,Yankees
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Rays
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Jays
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Orioles


In [24]:
def get_area(team):
    for each in list(mlb_cities.index.values):
        if team in each:
            return mlb_cities.at[each,"Metropolitan area"]

In [25]:
mlb_df["area"] =mlb_df["area"].apply(lambda x: get_area(x))

In [26]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League,area
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Chicago
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore


In [27]:
mlb_df.at[0,"area"]="Boston"

In [28]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League,area
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Boston
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore


In [29]:
out = []

In [38]:
for group , frame in mlb_df.groupby("area"):
    total_wins = np.sum(pd.to_numeric(frame["W"]))
    total_losses = np.sum(pd.to_numeric(frame["L"]))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_losses
    out_dict = {
        "Area":group,
        "Ratio":ratio
    }
    out.append(out_dict)

In [40]:
new_df = pd.DataFrame(out)

In [41]:
new_df.head()

,Area,Ratio
0,Atlanta,1.250000
1,Baltimore,0.408696
2,Boston,2.000000
3,Chicago,0.934524
4,Cincinnati,0.705263


In [42]:
new_df = new_df.set_index("Area")

In [43]:
new_df.head()

,Ratio
Area,
Atlanta,1.250000
Baltimore,0.408696
Boston,2.000000
Chicago,0.934524
Cincinnati,0.705263


In [44]:
out_df = pd.merge(new_df,population,how="inner",left_index=True,right_index=True)

In [45]:
out_df.head()

,Ratio,Population (2016 est.)[8]
Atlanta,1.250000,5789700
Baltimore,0.408696,2798886
Boston,2.000000,4794447
Chicago,0.934524,9512999
Cincinnati,0.705263,2165139


In [46]:
out_df["Population (2016 est.)[8]"] = pd.to_numeric(out_df["Population (2016 est.)[8]"])

In [47]:
out_df.head()

,Ratio,Population (2016 est.)[8]
Atlanta,1.250000,5789700
Baltimore,0.408696,2798886
Boston,2.000000,4794447
Chicago,0.934524,9512999
Cincinnati,0.705263,2165139


In [48]:
def mlb_correlation():
    population_by_region = []
    win_loss_by_region = []
    population_by_region = out_df["Population (2016 est.)[8]"].to_list()
    win_loss_by_region = out_df["Ratio"].to_list()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your List must be the Same Length."
    assert len(population_by_region) == 26, "Q3:There should be 26 Teams being Analysed for MLB"
    
    return np.float64(stats.pearsonr(population_by_region,win_loss_by_region)[0])

In [50]:
mlb_correlation()

0.10579597914829063

In [51]:
def get_mlb_data():
    return out_df

In [53]:
get_mlb_data()

,Ratio,Population (2016 est.)[8]
Atlanta,1.250000,5789700
Baltimore,0.408696,2798886
Boston,2.000000,4794447
Chicago,0.934524,9512999
Cincinnati,0.705263,2165139
Cleveland,1.281690,2055612
Dallas–Fort Worth,0.705263,7233323
Denver,1.263889,2853077
Detroit,0.653061,4297617
Houston,1.745763,6772470
